# From ANN to ONN





Given a keras file (neural network) and calibration data, build and train the photonic equivalent.

## Read calibration data

In [1]:
import numpy as np

# synthetic data
#calib_data=np.random.uniform(-1., +1., (100, 256))

# realistic data
N = 16
nb_data = 100
from ANN import get_db
(train_X, train_Y), (test_X, test_Y) = get_db("MNIST", N, shuffle=True)
calib_data=train_X[:nb_data]


print(calib_data.shape)
print(np.argmax(train_Y[:nb_data],axis=1))

2023-02-13 15:34:28.761283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pierrick/program/lzma/:/home/pierrick/program/python3_photon//install/Python-3.8.12/lib/:
2023-02-13 15:34:28.761358: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-13 15:34:29.698682: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pierrick/program/lzma/:/home/pierrick/program/python3_photon//install/Python-3.8.12/lib/:
2023-02-13 15:34:29.698805: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load

(60000, 256)
(60000, 10)
(10000, 256)
(10000, 10)
(100, 256)
[7 2 4 9 1 6 6 7 4 5 9 8 5 1 9 0 8 8 7 1 8 5 8 5 5 5 6 1 0 6 3 8 4 1 0 3 6
 3 7 2 3 6 4 6 9 5 7 5 6 6 0 7 9 0 2 1 6 9 5 2 1 3 6 8 2 4 9 0 3 0 4 4 4 6
 9 3 7 8 7 1 7 5 8 1 3 2 0 6 9 4 1 1 9 2 8 6 3 7 8 1]


## Read weights

In [ ]:
import keras
keras_path="tmp/MNIST/model_0/"
ann = keras.models.load_model(keras_path)
W1=ann.layers[1].get_weights()[0]
W2=ann.layers[3].get_weights()[0]

# free memory
keras.backend.clear_session()
del ann

## Teacher predictions

In [ ]:
Y1=[]
A=[]
Y2=[]

for x in calib_data:
    y1=np.dot(W1.T, x.reshape(len(x),1))# reshape for giving it to matrix multiplication
    a=np.maximum(y1,0)
    y2=np.dot(W2.T, a)
    
    Y1.append(y1)
    A.append(a)
    Y2.append(y2)
    
Y1=np.array(Y1)
A=np.array(A)
Y2=np.array(Y2)

In [ ]:
np.mean(np.argmax(Y2,axis=1).squeeze()==np.argmax(train_Y[:len(Y2)].squeeze(),axis=1))

## Train the student to mimic the teacher

In [ ]:
from TeacherStudent import TeacherStudent, TeacherMatrix

# Teacher
teacher_W1=TeacherMatrix(W1)
teacher_W2=TeacherMatrix(W2)

# Student
import ONN
hp={"lr":0.1, "lr_decay":10., "layers":[W1.shape[1]], "pattern": ["triangle"], "col_layer_limit": [8]}
student_W1=ONN.ONN(hp,{},{"epochs":5, "loss":ONN.MSE, "metrics":ONN.MSE})

hp={"lr":0.1, "lr_decay":10., "layers":[W2.shape[1]], "pattern": ["triangle"], "col_layer_limit": [8]}
student_W2=ONN.ONN(hp,{},{"epochs":5, "loss":ONN.MSE, "metrics":ONN.MSE})


teacher_student_W1=TeacherStudent(teacher_W1, student_W1)
teacher_student_W2=TeacherStudent(teacher_W2, student_W2)

In [ ]:
teacher_student_W1.fit(calib_data)
teacher_student_W2.fit(A.squeeze())

### Prediction with the student models (ONN)

In [ ]:
Y1_ONN=[]
A_ONN=[]
Y2_ONN=[]

for x in calib_data:
    y1=student_W1.predict(x)
    a=np.maximum(x,0)
    y2=student_W2.predict(x)
    
    Y1_ONN.append(y1)
    A_ONN.append(a)
    Y2_ONN.append(y2)
Y1_ONN=np.array(Y1_ONN)
A_ONN=np.array(A_ONN)
Y2_ONN=np.array(Y2_ONN)

In [ ]:
np.mean(np.argmax(Y2_ONN,axis=1).squeeze()==np.argmax(train_Y[:len(Y2_ONN)].squeeze(),axis=1))